In [ ]:
import os

import numpy as np
import pandas as pd

import coincidence_v4
from calibrations import calibration_20250303, itof_calibration


def process_file(file, save_path, calibration=calibration_20250303, symmetrize=False, itof_calibration=itof_calibration,
                 center=None, angle=None, ignore_itof=False):
    x, y, t, etof, itof = coincidence_v4.load_file(file) if not ignore_itof else coincidence_v4.load_file(file,
                                                                                                          coincidence=False)
    etof += 0.26 * np.random.random_sample(len(etof))
    match center, angle:
        case None, None:
            px, py, pz = calibration(x, y, etof, symmetrize=symmetrize)
        case (None, _):
            px, py, pz = calibration(x, y, etof, symmetrize=symmetrize, angle=angle)
        case (_, None):
            px, py, pz = calibration(x, y, etof, symmetrize=symmetrize, center=center)
        case _:
            px, py, pz = calibration(x, y, etof, symmetrize=symmetrize, center=center, angle=angle)

    if not ignore_itof:
        data = pd.DataFrame(
                {'px': px, 'py': py, 'pz': pz, 'm/q': itof_calibration(itof), 'raw_x': x, 'raw_y': y, 'raw_t': t,
                 'raw_etof': etof, 'raw_itof': itof})
    else:
        data = pd.DataFrame({'px': px, 'py': py, 'pz': pz, 'raw_x': x, 'raw_y': y, 'raw_t': t, 'raw_etof': etof})
    data.to_hdf(save_path, key='data', mode='w')


def process_folder(folder, save_folder=None, calibration=calibration_20250303, symmetrize=False,
                   itof_calibration=itof_calibration, center=None, angle=None, ignore_itof=False):
    if save_folder is None:
        save_folder = os.path.join(folder, 'calibrated')
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    for file in os.listdir(folder):
        if file.endswith('.cv4'):
            print(f'Processing {file}')
            save_path = os.path.join(save_folder, file.replace('.cv4', '_calibrated.h5'))
            process_file(os.path.join(folder, file), save_path, calibration, symmetrize, itof_calibration, center,
                         angle, ignore_itof)
            print(f'Saved to {save_path}')

In [ ]:
from calibrations import calibration_xe_800

fname = r"J:\ctgroup\Edward\DATA\VMI\20220613\xe005_e.cv4"
# process_file(fname, fname.replace('.cv4', '_calibrated.h5'))

process_file(fname, save_path=fname.replace(".cv4", "_calibrated.h5"), calibration=calibration_xe_800, symmetrize=False,
             ignore_itof=True)